In the previous section, we learned how to minimize a cost function by taking a step into the opposite direction of a gradient that is calculated from the whole training set; this is why this approach is sometimes also referred to as *batch* gradient descent. Now imagine we have a very large dataset with millions of data points, which is not uncommon in many machine learning applications. Running batch gradient descent can be computationally quite costly in such scenarios since we need to reevaluate the whole training dataset each time we take one step towards the global minimum.  

A popular alternative to the batch gradient descent algorithm is *stochastic gradient descent*, sometimes also called *iterative* or *on-line* gradient descent. Instead of updating the weights based on the sum of the accumulated errors over all samples $\mathbf{x}^{(i)}$:  
$$\Delta \mathbf{w} = \eta\sum_i(y^{(i)}-\phi(z^{(i)}))\mathbf{x}^{(i)}$$  
We update the weights incrementally* for each training sample:
$$\eta(y^{(i)}-\phi(z^{(i)}))\mathbf{x}^{(i)}$$  
incremental: If you are making incremental progress in math, you are moving slowly but steadily forward. Incremental describes regular, measurable movements that are usually small.  

Stochastic Gradient Descent:  
* it typically reaches convergence much faster because of the more frequent weight updates.  
* Since each gradient is calculated based on a single training example, the error surface is noisier than in gradient descent, which can also have the advantage that stochastic gradient descent can escape shallow local minima more readily. 
* Shuffling the training set for every epoch to prevent cycles.  
* Another advantage is we can use it for *online learning*. In online learning, our model is trained on-the-fly as new training data arrives.

In pseudocode, stochastic gradient descent can be presented as follows:  
1. Choose an initial vector of parameters $w$ and learning rate $\eta$  
2. Repeat until an approximate minimum is obtained:  
     1. Randomly shuffle examples in the training set  
     2. For $i = 1, 2, ..., n:$  
        $w:=w-\eta\nabla Q_i(w)$

In [1]:
from numpy.random import seed

In [ ]:
class AdalineSGD(object):
    """ADAptive LInear NEuron classifier.

    Parameters
    ------------
    eta : float
        Learning rate (between 0.0 and 1.0)
    n_iter : int
        Passes over the training dataset.

    Attributes
    -----------
    w_ : 1d-array
        Weights after fitting.
    cost_ : list
        Sum-of-squares cost function value averaged over all
        training samples in each epoch.
    shuffle : bool (default: True)
        Shuffles training data every epoch if True to prevent cycles.
    random_state : int (default: None)
        Set random state for shuffling and initializing the weights.
        
    """
    def __init__(self, eta=0.01, n_iter=10, shuffle=True, random_state=None):
        self.eta = eta
        self.n_iter = n_iter
        self.w_initialized = False
        self.shuffle = shuffle
        if random_state:
            seed(random_state)
    
    def fit(self, X, y):
        """ Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        y : array-like, shape = [n_samples]
            Target values.

        Returns
        -------
        self : object

        """
        self._initialize_weights(X.shape[1])
        self.cost_ = []
        for i in range(self.n_iter):
            if self.shuffle:
                X, y = self._shuffle(X, y)
            cost = []
            for xi, target in zip(X, y):
                cost.append(self._update_weights(xi, target))
            avg_cost = sum(cost) / len(y)
            self.cost_.append(avg_cost)